In [1]:
import os

# Lista contenido de data/raw/
raw_dir = r'C:\Users\CHRISTIAN\Documents\EvaluacionEjerciciosIA\data\raw' 
for dataset in os.listdir(raw_dir):
    dataset_path = os.path.join(raw_dir, dataset)
    if os.path.isdir(dataset_path):
        print(f"Dataset: {dataset}")
        for file in os.listdir(dataset_path):
            print(f"  - {file}")

Dataset: physical_exercise_dataset
  - 3d_distances.csv
  - angles.csv
  - labels.csv
  - landmarks.csv
  - v1.0
  - xyz_distances.csv
Dataset: videos
  - plank_video
  - pruebas
  - push_up_video
  - sit_up_video
  - squat_video


Inspeccion physical_exercise_dataset

In [2]:
import pandas as pd

# Ruta al dataset
dataset_path = r'C:\Users\CHRISTIAN\Documents\EvaluacionEjerciciosIA\data\raw\physical_exercise_dataset'

# Lee los archivos CSV
labels_df = pd.read_csv(dataset_path + '\labels.csv')
landmarks_df = pd.read_csv(dataset_path + '\landmarks.csv')
angles_df = pd.read_csv(dataset_path + r'\angles.csv')

# Muestra las primeras filas y columnas
print("Labels:")
print(labels_df.head())
print("\nLandmarks:")
print(landmarks_df.head())
print("\nAngles:")
print(angles_df.head())

Labels:
   pose_id                pose
0        0  jumping_jacks_down
1        1  jumping_jacks_down
2        2  jumping_jacks_down
3        3  jumping_jacks_down
4        4  jumping_jacks_down

Landmarks:
   pose_id    x_nose     y_nose      z_nose  x_left_eye_inner  \
0        0 -5.889507 -57.637520  -45.019750         -4.656085   
1        1 -4.255504 -62.935925 -128.907500         -2.977403   
2        2 -2.878917 -61.709988 -137.453340         -1.619050   
3        3 -4.242575 -60.371220 -135.094830         -3.118133   
4        4 -0.805543 -56.178570  -41.124413         -0.055174   

   y_left_eye_inner  z_left_eye_inner  x_left_eye  y_left_eye  z_left_eye  \
0        -62.832863        -44.571823   -3.302626  -63.386856  -44.567863   
1        -67.035990       -124.258545   -2.215265  -67.198250 -124.263240   
2        -65.693750       -132.181660   -0.785822  -65.814340 -132.184070   
3        -64.416000       -129.995930   -2.369744  -64.603290 -130.003400   
4        -58.50130

Explora Real-Time Exercise Recognition Dataset

Filtra y normaliza los datos de physical_exercise_dataset

In [3]:
import pandas as pd
import numpy as np

# Carga los datos
dataset_path = r'C:\Users\CHRISTIAN\Documents\EvaluacionEjerciciosIA\data\raw\physical_exercise_dataset'
labels_df = pd.read_csv(dataset_path + '\labels.csv')
landmarks_df = pd.read_csv(dataset_path + '\landmarks.csv')
angles_df = pd.read_csv(dataset_path + r'\angles.csv')

# Filtra los ejercicios relevantes
relevant_poses = ['pushups_down', 'pushups_up', 'situp_down', 'situp_up', 'squats_down', 'squats_up']
filtered_labels = labels_df[labels_df['pose'].isin(relevant_poses)]
pose_ids = filtered_labels['pose_id'].values

# Filtra landmarks y ángulos para los pose_ids relevantes
filtered_landmarks = landmarks_df[landmarks_df['pose_id'].isin(pose_ids)]
filtered_angles = angles_df[angles_df['pose_id'].isin(pose_ids)]

# Mapeo completo de 17 keypoints compatibles con YOLOv8s-pose (COCO)
keypoint_names = ['nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner', 'right_eye',
                 'right_eye_outer', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow',
                 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee',
                 'left_ankle', 'right_ankle']  # 17 keypoints completos
coco_indices = list(range(17))  # Índices 0-16

# Extrae 17 keypoints (x, y; ignoramos z por ahora)
keypoint_cols = [f'x_{name}' for name in keypoint_names] + [f'y_{name}' for name in keypoint_names]
keypoints_17 = filtered_landmarks[['pose_id'] + keypoint_cols].copy()

# Normaliza coordenadas (usamos 640x480 como tamaño de frame)
frame_width, frame_height = 640, 480
keypoints_17.loc[:, keypoint_cols] = keypoints_17[keypoint_cols] / np.array([frame_width, frame_height] * len(keypoint_names))

# Corrige ángulos (sin división por 10^6)
angle_cols = filtered_angles.columns[1:]
filtered_angles.loc[:, angle_cols] = filtered_angles[angle_cols]

# Guarda los datos procesados
keypoints_17.to_csv(r'C:\Users\CHRISTIAN\Documents\EvaluacionEjerciciosIA\data\processed\physical_exercise_keypoints_17.csv', index=False)
filtered_angles.to_csv(r'C:\Users\CHRISTIAN\Documents\EvaluacionEjerciciosIA\data\processed\physical_exercise_angles_normalized.csv', index=False)
filtered_labels.to_csv(r'C:\Users\CHRISTIAN\Documents\EvaluacionEjerciciosIA\data\processed\physical_exercise_labels.csv', index=False)

print(f"Procesados {len(filtered_labels)} frames con ejercicios relevantes.")
print("Keypoints, ángulos y etiquetas normalizados guardados.")

Procesados 45454 frames con ejercicios relevantes.
Keypoints, ángulos y etiquetas normalizados guardados.


código de enriquecimiento

In [4]:
import pandas as pd
import numpy as np
import sys
import os

# Añade el directorio src al path
sys.path.append(os.path.abspath('../src'))

from utils import calculate_angle

# Carga los datos procesados
keypoints_17 = pd.read_csv(r'C:\Users\CHRISTIAN\Documents\EvaluacionEjerciciosIA\data\processed\physical_exercise_keypoints_17.csv')
filtered_angles = pd.read_csv(r'C:\Users\CHRISTIAN\Documents\EvaluacionEjerciciosIA\data\processed\physical_exercise_angles_normalized.csv')
labels_df = pd.read_csv(r'C:\Users\CHRISTIAN\Documents\EvaluacionEjerciciosIA\data\processed\physical_exercise_labels.csv')  # Carga las etiquetas

# Une las etiquetas con los keypoints (usando pose_id como clave)
enriched_data = pd.merge(keypoints_17, labels_df[['pose_id', 'pose']], on='pose_id', how='left')

# Añade columnas para ángulos recalculados
angles_recalculated = pd.DataFrame(index=enriched_data.index, columns=['right_elbow_angle', 'left_elbow_angle',
                                                                     'right_knee_angle', 'left_knee_angle'])

for idx in enriched_data.index:
    right_shoulder = [enriched_data.loc[idx, 'x_right_shoulder'], enriched_data.loc[idx, 'y_right_shoulder']]
    right_elbow = [enriched_data.loc[idx, 'x_right_elbow'], enriched_data.loc[idx, 'y_right_elbow']]
    right_wrist = [enriched_data.loc[idx, 'x_right_wrist'], enriched_data.loc[idx, 'y_right_wrist']]
    left_shoulder = [enriched_data.loc[idx, 'x_left_shoulder'], enriched_data.loc[idx, 'y_left_shoulder']]
    left_elbow = [enriched_data.loc[idx, 'x_left_elbow'], enriched_data.loc[idx, 'y_left_elbow']]
    left_wrist = [enriched_data.loc[idx, 'x_left_wrist'], enriched_data.loc[idx, 'y_left_wrist']]
    right_hip = [enriched_data.loc[idx, 'x_right_hip'], enriched_data.loc[idx, 'y_right_hip']]
    right_knee = [enriched_data.loc[idx, 'x_right_knee'], enriched_data.loc[idx, 'y_right_knee']]
    right_ankle = [enriched_data.loc[idx, 'x_right_ankle'], enriched_data.loc[idx, 'y_right_ankle']]
    left_hip = [enriched_data.loc[idx, 'x_left_hip'], enriched_data.loc[idx, 'y_left_hip']]
    left_knee = [enriched_data.loc[idx, 'x_left_knee'], enriched_data.loc[idx, 'y_left_knee']]
    left_ankle = [enriched_data.loc[idx, 'x_left_ankle'], enriched_data.loc[idx, 'y_left_ankle']]

    angles_recalculated.loc[idx, 'right_elbow_angle'] = calculate_angle(right_shoulder, right_elbow, right_wrist)
    angles_recalculated.loc[idx, 'left_elbow_angle'] = calculate_angle(left_shoulder, left_elbow, left_wrist)
    angles_recalculated.loc[idx, 'right_knee_angle'] = calculate_angle(right_hip, right_knee, right_ankle)
    angles_recalculated.loc[idx, 'left_knee_angle'] = calculate_angle(left_hip, left_knee, left_ankle)

# Combina todos los datos
enriched_data = pd.concat([enriched_data, angles_recalculated], axis=1)
enriched_data.to_csv(r'C:\Users\CHRISTIAN\Documents\EvaluacionEjerciciosIA\data\processed\physical_exercise_enriched.csv', index=False)

print("\nDatos enriquecidos con ángulos recalculados y etiquetas guardados en physical_exercise_enriched.csv")
print("\nPrimeras 5 filas de enriched_data:")
print(enriched_data.head())


Datos enriquecidos con ángulos recalculados y etiquetas guardados en physical_exercise_enriched.csv

Primeras 5 filas de enriched_data:
   pose_id    x_nose  x_left_eye_inner  x_left_eye  x_left_eye_outer  \
0      659  0.005001          0.011026    0.010531          0.017030   
1      660  0.005643          0.009842    0.009092          0.018570   
2      661 -0.080741         -0.111531   -0.083575         -0.111369   
3      662  0.011821          0.024027    0.020280          0.030158   
4      663 -0.004911         -0.006103    0.000368          0.007008   

   x_right_eye_inner  x_right_eye  x_right_eye_outer  x_left_ear  x_right_ear  \
0           0.000794    -0.002900          -0.005019    0.022900    -0.011029   
1           0.001104    -0.001759          -0.003827    0.018725    -0.009088   
2          -0.083769    -0.111719          -0.083855   -0.107371    -0.081137   
3           0.011744     0.013145           0.008056    0.035715     0.004197   
4          -0.019879    -